In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [113]:
df = pd.read_csv('PSCompPars_data.csv')
df2 = pd.read_csv('webscraped_data.csv')

In [114]:
merged_df = pd.merge(df, df2[['pl_name', 'pl_fullname', 'pl_type']], on='pl_name', how='left')

# weird 6 rows don't have a mapping even though they still show in NASA Exoplanet Archive
merged_df = merged_df.dropna(subset=['pl_type']).reset_index(drop=True)

planet_types = {'Gas Giants':'Gas Giant',
                'Neptune-likes':'Neptune-like',
                'Super Earths':'Super-Earth',
                'Terrestrials':'Terrestrial',
                'Unknowns':'Unknown'}
merged_df['pl_type'] = merged_df['pl_type'].replace(planet_types)

In [73]:
merged_df.head(5)

,pl_name,disc_method,pl_controv_flag,pl_orbper_days,pl_radius_earth,pl_mass_earth,pl_eccentricity,pl_equil_tp,st_teff,st_radius,st_mass,st_acc,Ks_brightness,pl_fullname,pl_type
0,11 Com b,Radial Velocity,0,323.21000,12.2,4914.89849,0.238,NaN,4874.0,13.76,2.09,2.45,2.282,11 Comae Berenices b,Gas Giant
1,11 UMi b,Radial Velocity,0,516.21997,12.3,4684.81420,0.080,NaN,4213.0,29.79,2.78,1.93,1.939,11 Ursae Minoris b,Gas Giant
2,14 And b,Radial Velocity,0,186.76000,13.1,1131.15130,0.000,NaN,4888.0,11.55,1.78,2.55,2.331,14 Andromedae b,Gas Giant
3,14 Her b,Radial Velocity,0,1765.03890,12.6,2559.47216,0.373,NaN,5338.0,0.93,0.91,4.45,4.714,14 Herculis b,Gas Giant
4,16 Cyg B b,Radial Velocity,0,798.50000,13.5,565.73740,0.680,NaN,5750.0,1.13,1.08,4.36,4.651,16 Cygni B b,Gas Giant


In [115]:
mean_mass_by_type = merged_df.groupby('pl_type')['pl_mass_earth'].transform('mean')
merged_df['pl_mass_earth'] = merged_df['pl_mass_earth'].fillna(mean_mass_by_type)
mean_radius_by_type = merged_df.groupby('pl_type')['pl_radius_earth'].transform('mean')
merged_df['pl_radius_earth'] = merged_df['pl_radius_earth'].fillna(mean_radius_by_type)

print(merged_df['pl_mass_earth'].isna().sum())
print(merged_df['pl_radius_earth'].isna().sum())

6
6


In [116]:
dropped_rows = merged_df[merged_df['pl_radius_earth'].isna()]

In [120]:
merged_df = merged_df.dropna(subset=['pl_radius_earth'])

In [121]:
features = ['pl_mass_earth', 'pl_radius_earth']

# to use KNN to correct mislabeled data for planets where URL went to the Milky Way and recorded planet type as SPACE
X_train = merged_df[merged_df['pl_type'] != 'SPACE'][features]
y_train = merged_df[merged_df['pl_type'] != 'SPACE']['pl_type']
X_test = merged_df[merged_df['pl_type'] == 'SPACE'][features]

In [122]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [123]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

predicted_labels = knn.predict(X_test_scaled)

merged_df.loc[merged_df['pl_type'] == 'SPACE', 'pl_type'] = predicted_labels

In [ ]:
#knn.score(X_test_scaled, y)

In [124]:
data = pd.concat([merged_df, dropped_rows], ignore_index=True)

In [126]:
data

,pl_name,disc_method,pl_controv_flag,pl_orbper_days,pl_radius_earth,pl_mass_earth,pl_eccentricity,pl_equil_tp,st_teff,st_radius,st_mass,st_acc,Ks_brightness,pl_fullname,pl_type
0,11 Com b,Radial Velocity,0,323.21000,12.2,4914.89849,0.238,NaN,4874.0,13.76,2.09,2.45,2.282,11 Comae Berenices b,Gas Giant
1,11 UMi b,Radial Velocity,0,516.21997,12.3,4684.81420,0.080,NaN,4213.0,29.79,2.78,1.93,1.939,11 Ursae Minoris b,Gas Giant
2,14 And b,Radial Velocity,0,186.76000,13.1,1131.15130,0.000,NaN,4888.0,11.55,1.78,2.55,2.331,14 Andromedae b,Gas Giant
3,14 Her b,Radial Velocity,0,1765.03890,12.6,2559.47216,0.373,NaN,5338.0,0.93,0.91,4.45,4.714,14 Herculis b,Gas Giant
4,16 Cyg B b,Radial Velocity,0,798.50000,13.5,565.73740,0.680,NaN,5750.0,1.13,1.08,4.36,4.651,16 Cygni B b,Gas Giant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,KIC 10001893 c,Orbital Brightness Modulation,1,0.32528,NaN,NaN,NaN,NaN,27500.0,0.18,0.47,5.35,16.612,KIC 10001893 c,Unknown
5755,KIC 10001893 d,Orbital Brightness Modulation,1,0.81161,NaN,NaN,NaN,NaN,27500.0,0.18,0.47,5.35,16.612,KIC 10001893 d,Unknown
5756,LkCa 15 b,Imaging,1,NaN,NaN,NaN,NaN,NaN,4194.0,1.61,NaN,3.65,8.163,LkCa 15 b,Unknown
5757,LkCa 15 c,Imaging,1,NaN,NaN,NaN,NaN,NaN,4194.0,1.61,NaN,3.65,8.163,LkCa 15 c,Unknown


In [129]:
data = data[['pl_name', 'pl_fullname', 'pl_type']].reset_index(drop=True)
data

,pl_name,pl_fullname,pl_type
0,11 Com b,11 Comae Berenices b,Gas Giant
1,11 UMi b,11 Ursae Minoris b,Gas Giant
2,14 And b,14 Andromedae b,Gas Giant
3,14 Her b,14 Herculis b,Gas Giant
4,16 Cyg B b,16 Cygni B b,Gas Giant
...,...,...,...
5754,KIC 10001893 c,KIC 10001893 c,Unknown
5755,KIC 10001893 d,KIC 10001893 d,Unknown
5756,LkCa 15 b,LkCa 15 b,Unknown
5757,LkCa 15 c,LkCa 15 c,Unknown


In [131]:
data.to_csv('clean_labeled_data.csv')

In [ ]:
# add 6 Unknown rows back in before giving csv to Mash
# add visualization for clusters
# add visual for elbow method
# add evaluation 

In [74]:
merged_df['pl_controv_flag'].value_counts()

pl_controv_flag
0    5729
1      30
Name: count, dtype: int64

In [55]:
data = merged_df[['pl_name', 'pl_fullname', 'pl_type']]
data.to_csv('planet_data.csv', index=False)

In [46]:
merged_df['pl_type'].value_counts(dropna=False)

pl_type
Neptune-likes    1954
Gas Giants       1802
Super Earths     1729
Terrestrials      205
SPACE              63
NaN                 6
Unknowns            6
Name: count, dtype: int64